In [1]:
#   ___                  _                
#  / _/______ ____  ____(_)__ _______     
# / _/ __/ _ `/ _ \/ __/ (_-</ __/ _ \    
#/_//_/  \_,_/_//_/\__/_/___/\__/\___/    
# ___ _____(_)__ ___ ____  / /_(_)       
# / _ `/ __/ (_-</ _ `/ _ \/ __/ /        
# \_, /_/ /_/___/\_,_/_//_/\__/_/         
#/___/
#
#Samee Lab @ Baylor College Of Medicine
#francisco.grisanticanozo@bcm.edu

In [ ]:
library("Seurat")
library("scPred")
library("tidyverse")
require("SingleCellExperiment")
library("doParallel")

# Train model on training data

In [2]:
adata_train <- readRDS('../Data/Tabula_Sapiens/adata_train.rds')

In [ ]:
adata_train <- adata_train  %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

In [4]:
adata_train <- getFeatureSpace(adata_train, "celltype")

●  Extracting feature space for each cell type...
DONE!


In [5]:
cl <- makePSOCKcluster(20)
registerDoParallel(cl)
adata_train <- trainModel(adata_train, seed = 123,allowParallel = TRUE)
stopCluster(cl)

●  Training models for each cell type...
DONE!


In [6]:
get_scpred(adata_train)

'scPred' object
✔  Prediction variable = celltype 
✔  Discriminant features per cell type
✔  Training model(s)
Summary

|Cell type              |    n| Features|Method    |   ROC|  Sens|  Spec|
|:----------------------|----:|--------:|:---------|-----:|-----:|-----:|
|basal cell             | 2139|       50|svmRadial | 0.986| 0.899| 0.975|
|classical monocyte     | 1109|       50|svmRadial | 0.849| 0.313| 0.995|
|club cell              |  992|       50|svmRadial | 0.921| 0.467| 0.990|
|macrophage             | 3600|       50|svmRadial | 0.941| 0.794| 0.903|
|non-classical monocyte |  848|       50|svmRadial | 0.760| 0.116| 0.996|
|type ii pneumocyte     | 4144|       50|svmRadial | 0.950| 0.808| 0.953|

# Classify on predict data

In [7]:
adata_predict <- readRDS('../Data/Tabula_Sapiens/adata_predict.rds')

In [ ]:
adata_predict <- adata_predict %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA() 

In [9]:
adata_predict <- scPredict(adata_predict, adata_train)

●  Matching reference with new dataset...
	 ─ 2000 features present in reference loadings
	 ─ 2000 features shared between reference and new dataset
	 ─ 100% of features in the reference are present in new dataset
●  Aligning new data to reference...


Harmony 1/20

Harmony 2/20

Harmony 3/20

Harmony 4/20

Harmony 5/20

Harmony 6/20

Harmony 7/20

Harmony 8/20

Harmony 9/20

Harmony 10/20

Harmony 11/20

Harmony 12/20

Harmony 13/20

Harmony 14/20

Harmony converged after 14 iterations



●  Classifying cells...
DONE!


In [16]:
write.csv(adata_predict@meta.data,'../outputs/Tabula_Sapiens_predictions_scPRED.csv')